<p><img alt="Colaboratory logo" height="45px" src="https://www.quantreo.com/wp-content/uploads/2021/04/cropped-Logo_Quantreo_transparent.png" align="left" hspace="10px" vspace="0px"></p>

# Portfolio Optimization
In this section you will learn the concept of portfolio management.
The idea is to combine our strategies to reduce the risk of your investment. The notions we will study apply to strategies as well as to assets. You will be able to create your own portfolio using the algorithms that we will present.

<br>

### Content
* Sortino Optimization
* Min Variance Optimization
* Mean Variance Skewness Kurtosis Optimization

### environnement

In [ ]:
!pip install ta
!pip install yfinance

  Created wheel for ta: filename=ta-0.7.0-py3-none-any.whl size=28718 sha256=204927fff98990602bbf3350a36684cae5957873c1fa78bca54aa368d2b5c2fa
  Stored in directory: /root/.cache/pip/wheels/5e/74/e0/72395003bd1d3c8f3f5860c2d180ff15699e47a2733d8ebd38
Successfully built ta
     |████████████████████████████████| 6.3 MB 6.4 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=0aa7a37bcd6110dcca14e93565fbdc91d7df6c11351dd3248e9934093f93c594
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import pandas as pd
import ta
import yfinance as yf
from tqdm import tqdm
import seaborn as sns
from scipy.optimize import minimize

import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import cycler
colors = cycler('color',
                ['#669FEE', '#66EE91', '#9988DD',
                 '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('figure', facecolor='#313233')
plt.rc('axes', facecolor="#313233", edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors,
       labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor="#313233", edgecolor="#313233")
plt.rc("text", color="#C9C9C9")
plt.rc('figure', facecolor='#313233')

### Functions

In [ ]:
def RSI(val,neutral, window):
  """ 
        ------------------------------------------------------------------------------
        | Output: The function gives the returns of RSI strategy                     |
        ------------------------------------------------------------------------------
        | Inputs: -val (type dataframe pandas): Entry values of the stock            |
        |         -neutral (float): Value of neutrality, i.e. no action zone         |
        |         -window (float): rolling period for RSI                            |
        ------------------------------------------------------------------------------
  """

  # Print Error if there is no column Adj Close in the dataframe
  if "Adj Close" not in val.columns:
    ValueError("We need have a columns name Adj Close because all computation are about this column")
  
  val["rsi"] = ta.momentum.RSIIndicator(f["Adj Close"], window=window).rsi()


  
  """ Long buy Signal """
  # We put threshold
  overbuy = 70
  neutral_buy = 50 + neutral

  # Put nan values for the signal long columns
  val["signal_long"] = np.nan
  val["yersteday_rsi"] = f["rsi"].shift(1)
  # We need define the Open Long signal (RSI yersteday<55 and RSI today>55)
  val.loc[(val["rsi"]>neutral_buy) & (val["yersteday_rsi"]<neutral_buy), "signal_long"] = 1

  # We need define the Close Long signal (RSI yersteday>55 and RSI today<55) False signal
  val.loc[(val["rsi"]<neutral_buy)&(val["yersteday_rsi"]>neutral_buy), "signal_long"] = 0

  # We need define the Close Long signal (RSI yersteday>70 and RSI today<70) Over buy signal
  val.loc[(val["rsi"]<overbuy)&(val["yersteday_rsi"]>overbuy), "signal_long"] = 0



  """Short sell signal """
  # We put threshold
  oversell = 30
  neutral_buy = 50 - neutral

  # Put nan values for the signal short columns
  val["signal_short"] = np.nan
  val["yersteday_rsi"] = val["rsi"].shift(1)
  # We need define the Open Short signal (RSI yersteday>45 and RSI today<45)
  val.loc[(val["rsi"]<neutral_buy) & (val["yersteday_rsi"]>neutral_buy), "signal_short"] = -1

  # We need define the Close Short signal (RSI yersteday<45 and RSI today>45) False signal
  val.loc[(val["rsi"]>neutral_buy)&(val["yersteday_rsi"]<neutral_buy), "signal_short"] = 0

  # We need define the Close Short signal (RSI yersteday<30 and RSI today>30) Over sell signal
  val.loc[(val["rsi"]>oversell)&(val["yersteday_rsi"]<oversell), "signal_short"] = 0


  """ Compute the returns """
  # Compute the percentage of variation of the asset
  val["pct"] = val["Adj Close"].pct_change(1)

  # Compute the positions
  val["Position"] = (val["signal_short"].fillna(method="ffill")+ val["signal_long"].fillna(method="ffill"))

  # Compute the return of the strategy
  val["return"] = val["pct"]*(val["Position"].shift(1))

  return val["return"]

In [ ]:
def drawdown_function(serie):
  
  # We compute Cumsum of the returns
  cum = serie.dropna().cumsum()+1

  # We compute max of the cumsum on the period (accumulate max)
  running_max = np.maximum.accumulate(cum)

  # We compute drawdown
  drawdown  = cum/running_max - 1 
  return drawdown

In [ ]:
def BackTest(serie):

  # Import the benchmarcl
  sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
  sp500.columns = ["SP500"]

  val = pd.concat((serie, sp500), axis=1).dropna()
  
  # Compute the drawdown
  drawdown = drawdown_function(serie)
  max_drawdown = -np.min(drawdown)

  # Put a subplots
  fig, (cum, dra) = plt.subplots(1,2, figsize=(15,6))
  
  # Put a Suptitle
  fig.suptitle("Backtesting", size=20)

  # Put the cumsum
  cum.plot(serie.cumsum(), color="#39B3C7")
  cum.plot(val["Adj Close"].cumsum(), color="#B85A0F")
  cum.legend(["Portfolio", "SP500"])
  # Set individual title
  cum.set_title("Cumulative Return", size=13)

  # Put the drawdown
  dra.fill_between(drawdown.index,0,drawdown, color="#C73954", alpha=0.65)

  # Set individual title
  dra.set_title("Drawdown", size=13)

  # Plot the graph
  plt.show()

  # Compute the sortino
  sortino = np.sqrt(252)*serie.mean()/serie.loc[serie<0].std()

  # Compute the beta
  beta = np.cov(val,rowvar=False)[0][1] /np.var(val["Adj Close"].dropna())

  # Compute the alpha
  alpha = 252*serie.mean() - 252*beta*serie.mean()

  # Print the statistics
  print(f"Sortino: {np.round(sortino,3)}")
  print(f"Beta: {np.round(beta,3)}")
  print(f"Alpha: {np.round(alpha,3)}")
  print(f"MaxDrawdown: {np.round(max_drawdown*100,3)} %")


In [ ]:
def opt(f):
  # We set lists for the possible values of neutral and window
  neutral_values = [i*2 for i in range(10)]
  window_values = [i*2 for i in range(1,11)]

  # Set some Border of the datasets
  start_train, end_train = "2017-01-01", "2019-01-01"
  start_test, end_test = "2019-01-01", "2020-01-01"
  start_valid, end_valid = "2020-01-01", "2021-01-01"


  # Initialize the list
  resume = []

  # Loop to add the values in the list
  for i in range(len(neutral_values)):
      for j in range(len(window_values)):
        # Compute the returns
        return_train = RSI(f.loc[start_train:end_train], neutral_values[i], window_values[j])
        return_test = RSI(f.loc[start_test:end_test], neutral_values[i], window_values[j])

        # Compute the sortino
        sortino_train = np.sqrt(252) * return_train.mean() / return_train[return_train<0].std()
        sortino_test = np.sqrt(252) * return_test.mean() / return_test[return_test<0].std()

        # We do list of list to do a dataframe
        values = [neutral_values[i], window_values[j], sortino_train, sortino_test]
        resume.append(values)

  resume = pd.DataFrame(resume, columns=["Neutral", "Window", "Sortino Train", "Sortino test"])

  # Order by sortino
  ordered_resume = resume.sort_values(by="Sortino Train", ascending=False)



  for i in range(len(resume)):
    # Take the best
    best = ordered_resume.iloc[0+i:1+i,:]

    # Compute the sortino
    Strain = best["Sortino Train"].values[0]
    Stest = best["Sortino test"].values[0]

    # Take best neutral and best window
    best_neutral = best["Neutral"].values[0]
    best_window = best["Window"].values[0]

    # If the Sortino of the train and the test are good we stop the loop
    if Stest>0.5 and Strain>0.5:
      break

    # If there is no values enought good the put 0 in all values
    else:
      best_neutral = 0
      best_window = 0
      Strain = 0
      Stest = 0
  return [best_neutral, best_window, Strain, Stest]

# Section1:  Sortino Optimization

In [ ]:
# Import Yahoo name


# Initialize the lists



# Compute best parameters for each Asset


  # Import the asset

  # Put the values


In [ ]:
# Add asset columns to each list of resume


# Create a dataframe

# Indexing by asset


In [ ]:
# Order the dataframe using the Train sortino


In [ ]:
# Border of sets


# Create a dataframe to put the strategies (The assets of the portfolio)


  # Import the asset

  # Extract opptimal neutral

  # Exctract optimal window


In [ ]:
# Plot the strategies


In [ ]:
def SR_criterion(weight, returns):
  """ 
  ------------------------------------------------------------------------------
  | Output: Opposite Sortino ratio to do a m imization                         |
  ------------------------------------------------------------------------------
  | Inputs: -Weight (type ndarray numpy): Wheight for portfolio                |
  |         -returns (type dataframe pandas): Returns of stocks                |
  ------------------------------------------------------------------------------
  """
  pf_return = returns.values.dot(weight)
  mu = np.mean(pf_return) 
  sigma = np.std(pf_return[pf_return<0])
  Sortino = -mu/sigma
  return Sortino


# Compute the lenght of the strategies


# Initialisation weight value


# Optimization constraints problem



# Put the bounds


# Optimization problem solving


# Result for computations



In [ ]:
# BackTest Sortino Optimization


# Section 2: Min Variance Optimization

In [ ]:
def MV_criterion(weight,Returns_data):
  """ 
  ------------------------------------------------------------------------------
  | Output: optimization porfolio criterion                                    |
  ------------------------------------------------------------------------------
  | Inputs: -weight (type ndarray numpy): Wheight for portfolio                |
  |         -Returns_data (type ndarray numpy): Returns of stocks              |
  ------------------------------------------------------------------------------
  """
  portfolio_return=np.multiply(Returns_data,np.transpose(weight))
  portfolio_return=np.sum(portfolio_return,1)
  mean_ret=np.mean(portfolio_return,0)
  sd_ret=np.std(portfolio_return,0)
  criterion = sd_ret
  return criterion



# Compute the lenght of the strategies

# Initialisation weight value

# Optimization constraints problem

# Put the bounds

# Optimization problem solving

# Result for computations


In [ ]:
# BackTest MinVar Optimization


# Section3: Mean Variance Skewness Kurtosis Optimization

In [ ]:
def SK_criterion(weight,Returns_data):
  """ 
  ------------------------------------------------------------------------------
  | Output: optimization porfolio criterion                                    |
  ------------------------------------------------------------------------------
  | Inputs: -weight (type ndarray numpy): Wheight for portfolio                |
  |         -Returns_data (type ndarray numpy): Returns of stocks              |
  ------------------------------------------------------------------------------
  """
  from scipy.stats import skew, kurtosis

  Lambda_RA = 3
  portfolio_return=np.multiply(Returns_data,np.transpose(weight))
  portfolio_return=np.sum(portfolio_return,1)
  mean_ret=np.mean(portfolio_return,0)
  sd_ret=np.std(portfolio_return,0)
  skew_ret=skew(portfolio_return,0)
  kurt_ret=kurtosis(portfolio_return,0)
  W=1;
  Wbar=1*(1+0.25/100);
  criterion=np.power(Wbar,1-Lambda_RA)/(1+Lambda_RA)+np.power(Wbar,-Lambda_RA)*W*mean_ret-Lambda_RA/2*np.power(Wbar,-1-Lambda_RA)*np.power(W,2)*np.power(sd_ret,2)+Lambda_RA*(Lambda_RA+1)/(6)*np.power(Wbar,-2-Lambda_RA)*np.power(W,3)*skew_ret-Lambda_RA*(Lambda_RA+1)*(Lambda_RA+2)/(24)*np.power(Wbar,-3-Lambda_RA)*np.power(W,4)*kurt_ret
  criterion=-criterion;
  return criterion


# Compute the lenght of the strategies

# Initialisation weight value

# Optimization constraints problem

# Put the bounds

# Optimization problem solving

# Result for computations


In [ ]:
# Plot the performances
